In [1]:
import Pkg; Pkg.add(Pkg.PackageSpec(url="https://github.com/JuliaComputing/JuliaAcademyData.jl"))
using JuliaAcademyData; activate("Parallel_Computing")

   Updating git-repo `https://github.com/JuliaComputing/JuliaAcademyData.jl`


   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %

  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
 Activating environment at `~/.julia/packages/JuliaAcademyData/3C2GY/courses/Parallel_Computing/Project.toml`


# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can
lead to long discussions, first as to exactly what is being benchmarked and
secondly what explains the differences.  These simple questions can sometimes
get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for
yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT:
[Boxes and registers](https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb)).

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - Julia (built-in)
    - Julia (hand-written)
    - C (hand-written)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
- Towards exploiting parallelism with Julia
    - Allowing for floating point associativity
    - Making use of four cores at once: built-in
    - Making use of four cores at once: hand-written
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [2]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Array{Float64,1}:
 0.9164315962038878
 0.37301961556222496
 0.44705137703347275
 0.3342220772459985
 0.38398867852017937
 0.7219645648092967
 0.7302818832599542
 0.9114207992801229
 0.883009797805091
 0.5369992661389493
 0.3618269446580824
 0.7303833995943245
 0.2322321754236525
 ⋮
 0.7436360910305535
 0.5781368368196613
 0.17205257402855345
 0.5247763203639646
 0.8986867590964618
 0.1723772424582013
 0.8213696394389354
 0.44231131077081765
 0.9984650838001967
 0.9986306355098091
 0.522523990681141
 0.15007418305575326

In [3]:
sum(a)

4.99931302335025e6

The expected result is ~0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

In [4]:
@time sum(a)

  0.004601 seconds (1 allocation: 16 bytes)


4.99931302335025e6

In [5]:
@time sum(a)

  0.004008 seconds (1 allocation: 16 bytes)


4.99931302335025e6

In [6]:
@time sum(a)

  0.005939 seconds (1 allocation: 16 bytes)


4.99931302335025e6

The `@time` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a `BenchmarkTools.jl` package to make benchmarking easy and accurate:

In [7]:
using BenchmarkTools

In [8]:
@benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.520 ms (0.00% GC)
  median time:      5.707 ms (0.00% GC)
  mean time:        5.731 ms (0.00% GC)
  maximum time:     7.511 ms (0.00% GC)
  --------------
  samples:          871
  evals/sample:     1

# 1. Julia Built-in

So that's the performance of Julia's built-in sum — but that could be doing any number of tricks to be fast, including not using Julia at all in the first place! Of course, it is indeed written in Julia, but would it perform if we write a naive implementation ourselves?

In [9]:
@which sum(a)

sum(a::AbstractArray; dims) in Base at reducedim.jl:652

Let's save these benchmark results to a dictionary so we can start keeping track of them and comparing them down the line.

In [10]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.479 ms (0.00% GC)
  median time:      5.862 ms (0.00% GC)
  mean time:        5.974 ms (0.00% GC)
  maximum time:     8.063 ms (0.00% GC)
  --------------
  samples:          836
  evals/sample:     1

In [11]:
d = Dict()
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 1 entry:
  "Julia built-in" => 4.47907

# 2. Julia (hand-written)

In [12]:
function mysum(A)
    s = 0.0
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [13]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     10.139 ms (0.00% GC)
  median time:      10.956 ms (0.00% GC)
  mean time:        11.011 ms (0.00% GC)
  maximum time:     12.968 ms (0.00% GC)
  --------------
  samples:          454
  evals/sample:     1

In [14]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 2 entries:
  "Julia hand-written" => 10.1385
  "Julia built-in"     => 4.47907

So that's about 2x slower than the builtin definition. We'll see why later on.

But first: is this fast?  How would we know?  Let's compare it to some other languages...

#  3. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [15]:
using Libdl
C_code = """
    #include <stddef.h>
    double c_sum(size_t n, double *X) {
        double s = 0.0;
        for (size_t i = 0; i < n; ++i) {
            s += X[i];
        }
        return s;
    }
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [16]:
c_sum(a)

4.999313023350245e6

In [17]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

true

We can now benchmark the C code directly from Julia:

In [18]:
c_bench = @benchmark c_sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     10.088 ms (0.00% GC)
  median time:      10.878 ms (0.00% GC)
  mean time:        10.943 ms (0.00% GC)
  maximum time:     12.986 ms (0.00% GC)
  --------------
  samples:          457
  evals/sample:     1

In [19]:
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

Dict{Any,Any} with 3 entries:
  "C"                  => 10.0877
  "Julia hand-written" => 10.1385
  "Julia built-in"     => 4.47907

# 4. Python's built in `sum`

The `PyCall` package provides a Julia interface to Python:

In [20]:
using PyCall

In [21]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [22]:
pysum(a)

4.999313023350245e6

In [23]:
pysum(a) ≈ sum(a)

true

In [24]:
py_list_bench = @benchmark $pysum($a)

BenchmarkTools.Trial: 
  memory estimate:  352 bytes
  allocs estimate:  7
  --------------
  minimum time:     1.285 s (0.00% GC)
  median time:      1.312 s (0.00% GC)
  mean time:        1.313 s (0.00% GC)
  maximum time:     1.341 s (0.00% GC)
  --------------
  samples:          4
  evals/sample:     1

In [25]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any,Any} with 4 entries:
  "C"                  => 10.0877
  "Julia hand-written" => 10.1385
  "Julia built-in"     => 4.47907
  "Python built-in"    => 1285.5

# 5. Python: `numpy`

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [26]:
using Conda

In [27]:
# Conda.add("numpy")

In [28]:
numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at In[28]:1
└ @ Core In[28]:1


BenchmarkTools.Trial: 
  memory estimate:  352 bytes
  allocs estimate:  7
  --------------
  minimum time:     3.811 ms (0.00% GC)
  median time:      4.192 ms (0.00% GC)
  mean time:        4.255 ms (0.00% GC)
  maximum time:     5.483 ms (0.00% GC)
  --------------
  samples:          1174
  evals/sample:     1

In [29]:
numpy_sum(a)

4.999313023350249e6

In [30]:
numpy_sum(a) ≈ sum(a)

true

In [31]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any,Any} with 5 entries:
  "C"                  => 10.0877
  "Julia hand-written" => 10.1385
  "Python numpy"       => 3.81148
  "Julia built-in"     => 4.47907
  "Python built-in"    => 1285.5

# 6. Python, hand-written

In [32]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x7f833a787790>

In [33]:
py_hand = @benchmark $sum_py($a)

BenchmarkTools.Trial: 
  memory estimate:  352 bytes
  allocs estimate:  7
  --------------
  minimum time:     1.583 s (0.00% GC)
  median time:      1.611 s (0.00% GC)
  mean time:        1.612 s (0.00% GC)
  maximum time:     1.644 s (0.00% GC)
  --------------
  samples:          4
  evals/sample:     1

In [34]:
sum_py(a)

4.999313023350245e6

In [35]:
sum_py(a) ≈ sum(a)

true

In [36]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any,Any} with 6 entries:
  "C"                   => 10.0877
  "Julia hand-written"  => 10.1385
  "Python numpy"        => 3.81148
  "Python hand-written" => 1583.4
  "Julia built-in"      => 4.47907
  "Python built-in"     => 1285.5

# Summary so far

In [37]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Python numpy................3.8
Julia built-in..............4.5
C..........................10.1
Julia hand-written.........10.1
Python built-in..........1285.5
Python hand-written......1583.4


We seem to have three different performance classes here: The numpy and Julia
builtins are leading the pack, followed by the hand-written Julia and C
definitions. Those seem to be about 2x slower.  And then we have the much much
slower Python definitions over 100x slower.

# Exploiting parallelism with Julia

The fact that our hand-written Julia solution was almost an even multiple of
2x slower than the builtin solutions is a big clue: perhaps theres some sort
of 2x parallelism going on here?

(In fairness, there are ways to exploit parallelism in other languages, too,
but for brevity we won't cover them)

# 7. Julia (allowing floating point associativity)

The `for` loop

```julia
for a in A
    s += a
end
```

defines a very strict _order_ to the summation: Julia follows exactly what you
wrote and adds the elements of `A` to the result `s` in the order it iterates.
Since floating point numbers aren't associative, a rearrangement here would
change the answer — and Julia is loathe to give you different answer than
the one you asked for.

You can, however, tell Julia to relax that rule and allow for associativity
with the `@fastmath` macro. This might allow Julia to rearrange the sum in an
advantageous manner.

In [38]:
function mysum_fast(A)
    s = 0.0
    for a in A
        @fastmath s += a
    end
    s
end

mysum_fast (generic function with 1 method)

In [39]:
j_bench_hand_fast = @benchmark mysum_fast($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.538 ms (0.00% GC)
  median time:      5.749 ms (0.00% GC)
  mean time:        5.875 ms (0.00% GC)
  maximum time:     7.877 ms (0.00% GC)
  --------------
  samples:          850
  evals/sample:     1

In [40]:
mysum_fast(a)

4.999313023350276e6

In [41]:
d["Julia hand-written fast"] = minimum(j_bench_hand_fast.times) / 1e6
d

Dict{Any,Any} with 7 entries:
  "C"                       => 10.0877
  "Julia hand-written"      => 10.1385
  "Python numpy"            => 3.81148
  "Python hand-written"     => 1583.4
  "Julia built-in"          => 4.47907
  "Python built-in"         => 1285.5
  "Julia hand-written fast" => 4.53804

# 8. Distributed Julia (built-in)

We can take this one step further: nearly every modern computer these days has
multiple cores. All the above solutions are working one core hard, but all the
others are just sitting by idly. Let's put them to work!

In [49]:
#using Pkg
#Pkg.add("DistributedArrays")
using Distributed
using DistributedArrays
addprocs(4)
@sync @everywhere workers() include("/etc/julia/startup.jl")
@everywhere using DistributedArrays

In [56]:
adist = distribute(a)
j_bench_dist = @benchmark sum($adist)

BenchmarkTools.Trial: 
  memory estimate:  13.30 KiB
  allocs estimate:  364
  --------------
  minimum time:     3.614 ms (0.00% GC)
  median time:      4.677 ms (0.00% GC)
  mean time:        4.674 ms (0.00% GC)
  maximum time:     6.156 ms (0.00% GC)
  --------------
  samples:          1067
  evals/sample:     1

In [57]:
d["Julia 4x built-in"] = minimum(j_bench_dist.times) / 1e6
d

Dict{Any,Any} with 9 entries:
  "C"                       => 10.0877
  "Julia hand-written"      => 10.1385
  "Python numpy"            => 3.81148
  "Python hand-written"     => 1583.4
  "Julia built-in"          => 4.47907
  "Python built-in"         => 1285.5
  "Julia 4x built-in"       => 3.61403
  "Julia 4x hand-written"   => 4.25633
  "Julia hand-written fast" => 4.53804

# 9. Distributed Julia (hand-written)

Ok, that might be cheating, too — it's again just calling a library
function. Is it possible to write distributed sum ourselves?

In [58]:
function mysum_dist(a::DArray)
    r = Array{Future}(undef, length(procs(a)))
    for (i, id) in enumerate(procs(a))
        r[i] = @spawnat id sum(localpart(a))
    end
    return sum(fetch.(r))
end

mysum_dist (generic function with 1 method)

In [59]:
j_bench_hand_dist = @benchmark mysum_dist($adist)

BenchmarkTools.Trial: 
  memory estimate:  19.38 KiB
  allocs estimate:  542
  --------------
  minimum time:     3.718 ms (0.00% GC)
  median time:      5.209 ms (0.00% GC)
  mean time:        5.154 ms (0.00% GC)
  maximum time:     8.835 ms (0.00% GC)
  --------------
  samples:          967
  evals/sample:     1

In [60]:
d["Julia 4x hand-written"] = minimum(j_bench_hand_dist.times) / 1e6
d

Dict{Any,Any} with 9 entries:
  "C"                       => 10.0877
  "Julia hand-written"      => 10.1385
  "Python numpy"            => 3.81148
  "Python hand-written"     => 1583.4
  "Julia built-in"          => 4.47907
  "Python built-in"         => 1285.5
  "Julia 4x built-in"       => 3.61403
  "Julia 4x hand-written"   => 3.71844
  "Julia hand-written fast" => 4.53804

# Overall Summary

In [61]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia 4x built-in...........3.6
Julia 4x hand-written.......3.7
Python numpy................3.8
Julia built-in..............4.5
Julia hand-written fast.....4.5
C..........................10.1
Julia hand-written.........10.1
Python built-in..........1285.5
Python hand-written......1583.4


# Key take-aways

* Julia allows for serial C-like performance, even with hand-written functions
* Julia allows us to exploit many forms of parallelism to further improve performance. We demonstrated:
    * Single-processor parallelism with SIMD
    * Multi-process parallelism with DistributedArrays
* But there are many other ways to express parallelism, too!